In [1]:
from typing import List
import tfx.v1 as tfx
from tfx.dsl.component.experimental.decorators import component
from tfx.types import artifact, standard_artifacts

from module_file import _FEATURE_KEYS, _NUM_PARAM, _INPUT_DATA, _TARGET_DATA, SelectorFunc, ScoreFunc

In [2]:
class FeatureSelectionArtifact(artifact.Artifact):
  """Output artifact containing feature scores from the Feature Selection component"""
  TYPE_NAME = 'Feature Selection'
  PROPERTIES = {
      'scores': artifact.Property(type=artifact.PropertyType.JSON_VALUE),
      'p_values': artifact.Property(type=artifact.PropertyType.JSON_VALUE),
      'selected_features': artifact.Property(type=artifact.PropertyType.JSON_VALUE),
      'selected_data': artifact.Property(type=artifact.PropertyType.JSON_VALUE)
  }

In [3]:
# Select features based on scores
selector = SelectorFunc(ScoreFunc, k=3)
selected_data = selector.fit_transform(_INPUT_DATA, _TARGET_DATA).tolist()
selected_features = [val for (idx, val) in enumerate(_FEATURE_KEYS) if idx in selector.get_support(indices=True)]

# get scores and p-values for artifacts
selector_scores = selector.scores_
selector_p_values = selector.pvalues_

# merge scores and pvalues with feature keys to create a dictionary
selector_scores_dict = dict(zip(_FEATURE_KEYS, selector_scores))
selector_pvalues_dict = dict(zip(_FEATURE_KEYS, selector_p_values))
print(selected_features)

['sepal length (cm)', 'petal length (cm)', 'petal width (cm)']


In [4]:
@component
def FeatureSelection(feature_selection: tfx.dsl.components.OutputArtifact[FeatureSelectionArtifact],
                    ):
  """Feature Selection component"""

  # Select features based on scores
  selector = SelectorFunc(ScoreFunc, k=_NUM_PARAM)
  selected_data = selector.fit_transform(_INPUT_DATA, _TARGET_DATA).tolist()
  selected_features = [val for (idx, val) in enumerate(_FEATURE_KEYS) if idx in selector.get_support(indices=True)]

  # get scores and p-values for artifacts
  selector_scores = selector.scores_
  selector_p_values = selector.pvalues_

  # merge scores and pvalues with feature keys to create a dictionary
  selector_scores_dict = dict(zip(_FEATURE_KEYS, selector_scores))
  selector_pvalues_dict = dict(zip(_FEATURE_KEYS, selector_p_values))
    
  feature_selection.scores = selector_scores_dict
  feature_selection.p_values = selector_pvalues_dict
  feature_selection.selected_features = selected_features
  feature_selection.selected_data = selected_data

In [5]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
context = InteractiveContext()

In [6]:
feature_selector = FeatureSelection()

In [7]:
context.run(feature_selector)

ExecutionResult(
    component_id: FeatureSelection
    execution_id: 1
    outputs:
        feature_selection: Channel(
            type_name: Feature Scores and P-Values
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/var/folders/55/gk05zk9j6596lk4gv9p8sk680000gn/T/tfx-interactive-2021-07-29T04_22_31.091883-5eci0ni6/FeatureSelection/feature_selection/1"
        properties {
          key: "p_values"
          value {
            struct_value {
              fields {
                key: "petal length (cm)"
                value {
                  number_value: 5.533972277194025e-26
                }
              }
              fields {
                key: "petal width (cm)"
                value {
                  number_value: 2.7582496530034537e-15
                }
              }
              fields {
                key: "sepal length (cm)"
                value {
                  number_value: 0.004476514990225723
                }
              }
              fields {
                key: "sepal width (cm)"
                value {
                  number_value: 0.1563959804316254
                }
              }
            }
          }
        }
        properties {
          key: "scores"
          value {
            struct_value {
              fields {
                key: "petal length (cm)"
                value {
                  number_value: 116.3126130920702
                }
              }
              fields {
                key: "petal width (cm)"
                value {
                  number_value: 67.04836020011113
                }
              }
              fields {
                key: "sepal length (cm)"
                value {
                  number_value: 10.817820878494022
                }
              }
              fields {
                key: "sepal width (cm)"
                value {
                  number_value: 3.710728303532493
                }
              }
            }
          }
        }
        properties {
          key: "selected_data"
          value {
            struct_value {
              fields {
                key: "__value__"
                value {
                  list_value {
                    values {
                      list_value {
                        values {
                          number_value: 1.4
                        }
                        values {
                          number_value: 0.2
                        }
                      }
                    }
                    values {
                      list_value {
                        values {
                          number_value: 1.4
                        }
                        values {
                          number_value: 0.2
                        }
                      }
                    }
                    values {
                      list_value {
                        values {
                          number_value: 1.3
                        }
                        values {
                          number_value: 0.2
                        }
                      }
                    }
                    values {
                      list_value {
                        values {
                          number_value: 1.5
                        }
                        values {
                          number_value: 0.2
                        }
                      }
                    }
                    values {
                      list_value {
                        values {
                          number_value: 1.4
                        }
                        values {
                          number_value: 0.2
                        }
                      }
                    }
                    values {
                      list_value {
                        values {
                          number_value: 1.7
                        }
            